In [1]:
import sys
sys.path.insert(0,r"C:\Users\pc\Documents")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
## Lectura de datos     
base_oriente = pd.read_csv(r'C:\Users\pc\Downloads\BASE_ORIENTE.txt', sep=r'¡', engine='python')
base_oriente.head()

## Eliminación de variables: 
base_oriente2 = base_oriente[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO',
        'PROM_SUSC', 'CANAL_ENVIO', 'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

In [3]:
# Vista previa
base_oriente2.head()

,SESUSERV,SERVDESC,SESUSUSC,SESUFEIN,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,SUSCNITC,TIPIFICACION_CLIENTE,...,CANTIDAD,CALIFICACION_VECTOR_SERVICIO,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,REGIONAL,ESTRATO_AGRUPADO,DEPARTAMENTO_AGR,ANTIGUEDAD_DIAS,CANAL_PAGO
0,7245,INTERNET,17021551,17/12/2019,SANTANDER,2,82816,7,13843277,NUEVO,...,3,58,"25919,2",DIGITAL,Hogares,ORIENTE,3.0,SANTANDER,154,FISICO
1,7255,TELEFONÍA,17073594,15/01/2020,SANTANDER,3,73698,7,1053325942,NUEVO,...,3,0,"31442,83",DIGITAL,Hogares,ORIENTE,4.0,SANTANDER,125,SIN PAGO
2,7245,INTERNET,17094768,25/01/2020,SANTANDER,2,100192,7,13851944,NUEVO,...,3,67,"42279,38",DIGITAL,Hogares,ORIENTE,2.0,SANTANDER,115,FISICO
3,7245,INTERNET,17120033,06/02/2020,SANTANDER,1,48750,7,91274082,NUEVO,...,1,100,21582,DIGITAL,Hogares,ORIENTE,3.0,SANTANDER,103,DIGITAL
4,7255,TELEFONÍA,17026787,20/12/2019,NORTE DE SANTANDER,2,46364,7,60334585,NUEVO,...,3,43,"26292,6",DIGITAL,Hogares,ORIENTE,2.0,NORTE DE SANTANDER,151,FISICO


In [4]:
## Numero completo de clientes en la region
len(base_oriente2)

283874

In [5]:
## Eliminacion de clientes con datos NA
base_oriente3 = base_oriente2.dropna()

In [6]:
len(base_oriente3)

267047

In [7]:
## Group by por nit para calificacion
b4 = base_oriente3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_oriente4 = pd.merge(base_oriente3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_oriente4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_oriente4['y'] = pd.cut(x=base_oriente4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])


In [8]:
dataTypeSeries = base_oriente4.dtypes
dataTypeSeries

cat = base_oriente4.select_dtypes(include=['object','category'])
cuanti = base_oriente4.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))

In [9]:
cuanti.corr(method = 'pearson')

,SESUSERV,SESUSUSC,SESUCUSA,SESUSAPE,SESUCICL,SUMA,CANTIDAD,CALIFICACION_VECTOR_SERVICIO_x,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CALIFICACION_VECTOR_SERVICIO_y
SESUSERV,1.000000,0.477807,-0.090269,-0.084482,0.126677,-0.056527,-0.052700,-0.035902,0.200539,-0.097850,-0.044802
SESUSUSC,0.477807,1.000000,0.015808,-0.077285,-0.415823,-0.394247,-0.398937,-0.196309,0.110116,-0.305126,-0.195538
SESUCUSA,-0.090269,0.015808,1.000000,0.345363,-0.085103,-0.350364,-0.083458,-0.427992,-0.129795,0.090108,-0.416710
SESUSAPE,-0.084482,-0.077285,0.345363,1.000000,0.003713,-0.099973,-0.005754,-0.126690,-0.189596,0.223187,-0.130552
SESUCICL,0.126677,-0.415823,-0.085103,0.003713,1.000000,0.489526,0.499051,0.233278,-0.063362,0.189186,0.236156
SUMA,-0.056527,-0.394247,-0.350364,-0.099973,0.489526,1.000000,0.904784,0.725993,0.097413,0.039547,0.713550
CANTIDAD,-0.052700,-0.398937,-0.083458,-0.005754,0.499051,0.904784,1.000000,0.525645,0.057477,0.098324,0.510475
CALIFICACION_VECTOR_SERVICIO_x,-0.035902,-0.196309,-0.427992,-0.126690,0.233278,0.725993,0.525645,1.000000,0.127268,-0.029133,0.967642
ESTRATO_AGRUPADO,0.200539,0.110116,-0.129795,-0.189596,-0.063362,0.097413,0.057477,0.127268,1.000000,-0.434956,0.131524
ANTIGUEDAD_DIAS,-0.097850,-0.305126,0.090108,0.223187,0.189186,0.039547,0.098324,-0.029133,-0.434956,1.000000,-0.030107


In [10]:
base_oriente5=cuanti.join(cat)
print(base_oriente5.dtypes)

#base_oriente5.to_csv(r'C:\Users\pc\Desktop\base_oriente_process.csv')

#base_oriente5 = pd.read_csv(r'C:\Users\pc\Desktop\base_oriente_process.csv', sep=',', engine='python')

X = base_oriente5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 
        #'SESUSUSC',
        'DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12',
        #'REGIONAL',
        'PROM_SUSC', 'CANAL_ENVIO',
        # 'SUMA',
        #'CANTIDAD',
        #'CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y','DEPARTAMENTO_AGR'
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
       'SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

y = base_oriente5['y']

SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUSAPE                            int64
SESUCICL                            int64
SUMA                                int64
CANTIDAD                            int64
CALIFICACION_VECTOR_SERVICIO_x      int64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                     int64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
PROM_SUSC                           int32
CANAL_ENVIO                         int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [11]:
## Vista previa de las variables preliminares para clasificacion
X.head()

,SESUSERV,SERVDESC,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,TIPIFICACION_CLIENTE,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CANAL_PAGO
0,7245,5,3,2,82816,7,1,10832,0,1,3.0,154,1
1,7255,20,3,3,73698,7,1,27145,0,1,4.0,125,3
2,7245,5,3,2,100192,7,1,59638,0,1,2.0,115,1
3,7245,5,3,1,48750,7,1,4792,0,1,3.0,103,0
4,7255,20,2,2,46364,7,1,11512,0,1,2.0,151,1


In [12]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.828690  0.817157  0.822883   8335.0   8219.0  0.942795
1             0.846003  0.885335  0.865222  29704.0  31085.0  0.894274
2             0.710478  0.652007  0.679988  15371.0  14106.0  0.839165
avg / total   0.804298  0.807545  0.805306  53410.0  53410.0  0.912096
DecisionTreeClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.852167  0.733773  0.788551   8335.0   7177.0  0.946917
1             0.892972  0.769055  0.826394  29704.0  25582.0  0.898315
2             0.591206  0.794288  0.677864  15371.0  20651.0  0.842728
avg / total   0.799758  0.770811  0.777742  53410.0  53410.0  0.925260
RandomForestClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.859077  0.692621  0.766921   8335.0   6720.0  0.9

In [14]:
# Importancia de variables
model = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)

importance = model.feature_importances_
# Total importancia de variables
for i,v in enumerate(importance):
	print('Variable: %0d, Score: %.5f' % (i,v))

Variable: 0, Score: 0.02143
Variable: 1, Score: 0.00000
Variable: 2, Score: 0.00007
Variable: 3, Score: 0.60988
Variable: 4, Score: 0.00000
Variable: 5, Score: 0.00812
Variable: 6, Score: 0.00839
Variable: 7, Score: 0.05486
Variable: 8, Score: 0.00002
Variable: 9, Score: 0.00369
Variable: 10, Score: 0.00000
Variable: 11, Score: 0.10875
Variable: 12, Score: 0.18480


In [ ]:
## El mejor modelo de clasificacion fue el KNN, veamos el k optimo con corridas de k = 2 a k = 10

In [15]:
classifiers = [
    KNeighborsClassifier(2),
    KNeighborsClassifier(3),
    KNeighborsClassifier(4),
    KNeighborsClassifier(5),
    KNeighborsClassifier(6),
    KNeighborsClassifier(7),
    KNeighborsClassifier(8),
    KNeighborsClassifier(9),
    KNeighborsClassifier(10)
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.762652  0.880504  0.817352   8335.0   9623.0  0.928898
1             0.802035  0.926239  0.859674  29704.0  34304.0  0.876755
2             0.774438  0.477783  0.590971  15371.0   9483.0  0.816164
avg / total   0.787947  0.790039  0.775739  53410.0  53410.0  0.894140
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.828690  0.817157  0.822883   8335.0   8219.0  0.942795
1             0.846003  0.885335  0.865222  29704.0  31085.0  0.894274
2             0.710478  0.652007  0.679988  15371.0  14106.0  0.839165
avg / total   0.804298  0.807545  0.805306  53410.0  53410.0  0.912096
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score  support     pred       AUC
0             0.835122  0.840432  0.837768   8335.0   8388.0  0.95193

In [17]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
#X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
#'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_fin = X
X_ = StandardScaler().fit_transform(X_fin)

for train_index, test_index in kf.split(X_, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(7).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                      y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


[             precision    recall  f1-score  support     pred       AUC
0             0.754701  0.723874  0.738966   8373.0   8031.0  0.916203
1             0.805992  0.847404  0.826179  29778.0  31308.0  0.876792
2             0.609978  0.562488  0.585271  15259.0  14071.0  0.804616
avg / total   0.741951  0.746639  0.743681  53410.0  53410.0  0.893532,              precision    recall  f1-score  support     pred       AUC
0             0.771288  0.747522  0.759219   8373.0   8115.0  0.923021
1             0.805656  0.849621  0.827054  29778.0  31403.0  0.874129
2             0.612151  0.557310  0.583445  15259.0  13892.0  0.803108
avg / total   0.744984  0.750103  0.746822  53410.0  53410.0  0.895139,              precision    recall  f1-score  support     pred       AUC
0             0.786419  0.713723  0.748310   8373.0   7599.0  0.911912
1             0.766411  0.848076  0.805178  29778.0  32951.0  0.814105
2             0.558942  0.471066  0.511256  15259.0  12860.0  0.712495
avg

In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

array([[ 6115,   798,  1460],
       [  511, 25281,  3985],
       [ 1454,  5295,  8509]], dtype=int64)